## **演示1101：贝叶斯分类基本原理**

### **提出问题**
如何从一条短信内容判断它是垃圾短信(Spam)的可能性？  
假设已经有大量短信文本，并且已知每条短信是否垃圾短信。那么再给出一条新短信文本，如何根据已有短信信息对这条新短信做出判断？

### **分析问题**
* 朴素贝叶斯方法  
对于一个分类问题，给定样本特征$x$，样本属于类别$y$的概率，根据贝叶斯理论计算如下：$ P(y|x)= \dfrac{P(x|y) P(y)}{P(x)}$  
假设$x$维度为$N$(即：有$N$个特征)，$c$代表$y$的可能分类，其维度为$K$(即：$y$有$K$个分类，分别是$c_1,c_2,\cdots,c_k$);  
若提供一个$x$向量(该向量可视为是由特征$x_1,x_2,\cdots,x_n$组合而成)，则计算出该$x$向量所对应的$y$是类别$k$的可能性为：  
$ P(y=c_k|x)= \dfrac{P(x|y=c_k) * P(y=c_k)}{P(x)}=\dfrac{P(x_1|y=c_k, x_2 |y=c_k, \cdots, x_n|y=c_k) * P(y=c_k)}{P(x_1,x_2,\cdots,x_n)} $  
**朴素**是指，这$n$个特征在概率上彼此独立，即：  
$ P(x_1|y=c_k, x_2|y=c_k, \cdots, x_n|y=c_k)=P(x_1|y=c_k ) * P(x_2 |y=c_k )* \cdots * P(x_n | y=c_k) $  
根据全概率公式：  
$ \begin{aligned}
& P(x_1,x_2,\cdots, x_n )
\\ & =P(x_1,x_2,\cdots,x_n|y=c_1) * P(y=c_1 ) + P(x_1,x_2,\cdots,x_n|y=c_2) * P(y=c_2 )+ \\ &
\cdots+ P(x_1,x_2,\cdots,x_n| y=c_k) * P(y=c_k)
\\ \\& =\sum_{k=1}^K [P(x_1,x_2,\cdots,x_n|y=c_k) * P(y=c_k)]
\\ \\& =\sum_{k=1}^K [P(x_1|y=c_k) * P(x_2| y=c_k) * \cdots * P(x_n| y=c_k ) * P(y=c_k)]
\\ \\& =\sum_{k=1}^K [P(y=c_k) * \prod_{i=1}^n P(x_i|y=c_k)]
\end{aligned} $  
最终可得：  
$ \begin{aligned}
& P(y=c_k|x) \\ \\ &=
\frac{P(x_1|y=c_k) * P(x_2|y=c_k) * \cdots * P(x_n|y=c_k) * P(y=c_k)}{P(x_1) * P(x_2) * \cdots * P(x_n)} \\ \\&=
\frac{[\prod_{i=1}^N P(x_i|y=c_k)] * P(y=c_k)}
{\sum_{k=1}^K [P(y=c_k)*\prod_{i=1}^N P(x_i|y=c_k)]}
\end{aligned}$  
* 朴素贝叶斯如何用于文本信息分类：  
对于$m$条文本信息(假设全是英文)，如果已经知道它们一共分为$K$类(例如，分为垃圾信息和非垃圾信息两类)，一般的做法是：  
 * 建立一个词汇表：$Dict[N]$  
    1) 将每一条文本信息拆分成若干个单词  
    2) 将每个单词加入到词汇表中，如果词汇表中已经存在该单词，则只保留一个  
    3) 遍历所有文本，建立起完整的词汇表，假设其词汇总数为N，这就是Feature的维度  
 * 计算先验概率  
    1) 遍历所有文本信息的分类，计算从1~K，每个类别所占的比重(概率)：$Probability_{c=k}$ 。例如，1号类别占总数的20%，2号占15%，等等  
    2) 按照信息分类，在每一个分类下的所有文本信息中，先统计该类下的单词总数($N_{c=k}$)，然后针对词汇表中的每个单词，依次计算其占$N_{c=k}$ 的比重(概率)。从而得到：$Probability\_Dict[N_{c=k}]$  
    3) 计算出所有类别下，每个单词的出现概率：$Probability\_Dict[K,N]$  
 * 预测新信息的类别  
    1) 将新信息拆分成若干个单词  
    2) 建立一个数组：$Test\_Feature[N]$，对照单词表，如果某个单词在单词表位于第$index$个元素，则$Test\_Feature[index]=1$；如果不存在则设为0  
    3) 根据贝叶斯公式：$ P(y=c_k|x)=\dfrac{\sum{[\prod_{i=1}^N P(x_i | y=c_k)] * P(y=c_k)}}{\sum_{k=1}^K [P(y=c_k ) * \prod_{i=1}^N P(x_i|y=c_k)]} $ ，对于每个类别，其分母部分都是相同的，因此只需要比较分子部分，最大的那个即是其所属类别。  
    4) 将$Test\_Feature$与$Probability\_Dict[i,N]$求内积，然后乘以$Probability_{c=i}$，得到上式中分子部分(类别为$i$时)。  
    5) 将$i$从$0$~$K$分别计算，结果最大的那个就是所属类别

### **注意事项**
上述过程并不复杂，但是在实际计算时会产生两个问题：  
* 平滑处理
 * 如果词汇表中，某个单词(Feature)在某些分类下占总单词数的比重为0，例如：对于分类2，词汇表中的第8个单词"hello"从来没有出现过，那么$P(x_8|y=c_2)=0$，进而贝叶斯公式中的整个分子部分都为0。这就意味着，无论该文本中是否存在其它单词，都会使其属于2号分类的概率为0。这是不合理的
 * 因此，需要确保每个Feature在每个分类中出现的次数"大于0"。一般的做法是，采用下列公式计算该单词在该类别中的先验概率：  
$ P(x_i|y=c_k)=\dfrac{T_{x_i|c_k} + \alpha}{T_{c_k} + \lambda * \alpha} $
   * $T_{x_i|c_k}$ 表示第$i$个特征在第$k$个分类中的出现次数；$T_{c_k }$表示第$k$个分类中的单词总数；
   * $\lambda$为$x$的特征数量。对于文本分类来说，$\lambda$可取成单词表的容量。
   * $\alpha$称为平滑参数，若取值为1，则称为拉普拉斯平滑(Laplace smoothing)；若取值在0~1之间，则称为Lidstone smoothing
 * 采用平滑处理后，不会再出现概率为0的情形，并且仍然可以保证所有特征的概率之和仍为1
   * 例如，假设某个分类下的单词总数为$T_{c_k}=2000$，而词汇表共有500个单词，则可设置$\lambda=500,\alpha=1$
   * 如果有1个单词出现了2000次(意味着只有这一个单词出现了，其它单词都没有出现过)，那么这该单词出现的概率为 (2000+1)/(2000+500x1)=2001/2500
   * 词汇表中剩余的499个单词，每个出现的概率为：1/2500，共计499/2500。二者之和正好为1
* 对数概率运算
 * 贝叶斯公式的分子部分： $\prod_{i=1}^N P(x_i|y=c_k)$ 存在概率连续乘积的情况。如果$N$比较大(Feature较多)，则连乘将导致结果越来越小，最后超出了浮点数运算范围而无法继续计算
 * 观察下列事实： $ P_1 * P_2 * P_3 * \cdots * P_n=e^{log(P_1 )+log(P_2 )+log(P_3)+\cdots+log(P_n)}$ 该式子将小数的连乘转换成自然对数结果的和，这样，即使每项都不大，但却避免了乘积更小的问题。
 * 借助这一点，在计算概率连乘时，应采用对数概率形式：  
$ \begin{aligned}
&log[\prod_{i=1}^N P(x_i|y=c_k)]\\ \\&=log(P(x_1 | y=c_k )) + log(P(x_2 |y=c_k)) +\cdots+log(P(x_N|y=c_k)) \\ \\&
= \sum_{i=1}^N log(P(x_i|y=c_k))
\end{aligned}$  
$ \begin{aligned}
&P(y=c_k|x) \\ \\
&=\dfrac{[\prod_{i=1}^N P(x_i|y=c_k)] * P(y=c_k)}{\sum_{k=1}^K [P(y=c_k) * \prod_{i=1}^N P(x_i|y=c_k)]} \\ \\
&=\dfrac{P(y=c_k ) * e^{\sum_{i=1}^N log(P(x_i|y=c_k))}}{\sum_{k=1}^K [P(y=c_k ) * e^{\sum_{i=1}^N log(P(x_i|y=c_k))}]}
\end{aligned}$  
这将大大改善浮点运算的效能

### **案例1：构造词汇表**

In [1]:
''' 演示如何构造词汇表 '''

import numpy as np

# 模拟训练数据
def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],        # 第1个文本，已拆分成单词
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],    # 第2个文本
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],       # 第3个文本
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],             # 第4个文本
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'], # 第5个文本
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]          # 第6个文本
    classVec=[0, 1, 0, 1, 0, 1]                      # 上述每个文本对应的类型：1表示不当言论，0表示正常言论
    return postingList, classVec

# 构造词汇表
def createVocabList(dataSet):
    vocabSet = set([])                            # set具有保证元素唯一性的特点
    for document in dataSet:
        # 先从document中取出所有单词(去掉重复的)，然后再与之前的vocabSet合并(并且去掉重复性的单词)
        vocabSet = vocabSet | set(document)
    return list(vocabSet)

listPosts, listClasses = loadDataSet()
myVocabList = createVocabList(listPosts)
print("词汇表总所有元素：", myVocabList)
print("词汇表总长度：", len(myVocabList))

词汇表总所有元素： ['quit', 'has', 'not', 'so', 'ate', 'buying', 'dalmation', 'my', 'posting', 'problems', 'take', 'stop', 'him', 'love', 'help', 'worthless', 'how', 'park', 'steak', 'please', 'I', 'food', 'maybe', 'stupid', 'mr', 'licks', 'is', 'flea', 'dog', 'to', 'garbage', 'cute']
词汇表总长度： 32


### **案例2：将每个文本语句表示成特征向量形式**

In [2]:
# 将inputSet(也就是一个语句)拆分成多个单词，并生成一个Feature行，标记每个单词在词汇表中是否存在
def setOfWords2Vec(vocabList, inputSet):
    returnVec=np.zeros(len(vocabList))              # 每个元素对应vocabList中的一个单词              
    for word in inputSet:
        if word in vocabList:
            # inputSet中的某个单词存在vocabList中，则returnVec中对应单词位置元素值设为1
            returnVec[vocabList.index(word)] = 1      
        else: print('单词【%s】在词汇表中暂不存在，忽略!'% word)
    return returnVec


trainMat=[]    # 定义训练数据。每行数据代表一个语句的特征向量表达
for postinDoc in listPosts:
    trainMat.append(setOfWords2Vec(myVocabList, postinDoc))

for trainFeature in trainMat:
    print(trainFeature)

[0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1.
 0. 0. 0. 0. 1. 1. 0. 0.]
[0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0.
 1. 1. 0. 0. 0. 1. 0. 0.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1.
 0. 0. 0. 0. 1. 0. 0. 0.]


### **案例3：计算每个单词在每种文本类别中的先验概率**

In [3]:
# 执行训练(计算先验概率)，返回统计出来的几个先验概率值：
# p0Vec：正常语句中，每个单词占总单词数的比例(概率)
# p1Vec：不当语句中，每个单词占总单词数的比例(概率)
# pAbusive：不当语句占总语句的比例
def train(trainMatrix, trainCategory):
    # trainMatrix中每行都是一个数组（长度为单词表长度)，记录了一条语句中的每个单词是否在单词表中存在。
    # 如果存在，则数组中对应单词位置元素值为1，否则为0
    numTrainDocs = len(trainMatrix)                           # 语句的数量
    numWord = len(trainMatrix[0])                             # 单词表的容量
    pAbusive = sum(trainCategory) / len(trainCategory)        # 计算不当语句占所有语句的比例(p1)，注意不当语句类别值设为1
    # 初始化为1，实际上是考虑了拉普拉斯平滑中，分子要加Alpha(1)的情形
    p0Num = np.ones(numWord)                                  # 存放每个单词在正常语句中出现的次数
    p1Num = np.ones(numWord)                                  # 存放每个单词在不当语句中出现的次数
    # 分母部分先算上拉普拉斯平滑中应该追加的Lambda*Alpha，其中Lambda取单词表的容量，Alpha设为1
    p0Demon = numWord                                         # 存放正常语句中单词的总数
    p1Demon = numWord                                         # 存放不当语句中单词的总数
    for i in range(numTrainDocs):
        if trainCategory[i] == 0:                             # 正常语句
            p0Num += trainMatrix[i]                           # 向量相加(分别记录每个单词的数量)
            p0Demon += sum(trainMatrix[i])                    # 累加单词总数
        else:
            p1Num += trainMatrix[i]
            p1Demon += sum(trainMatrix[i])
    p0Vec= np.log(p0Num / p0Demon)                            # 正常语句中，对数形式的单词概率
    p1Vec= np.log(p1Num / p1Demon)                            # 不当语句中，对数形式的单词概率
    return p0Vec, p1Vec, pAbusive

p0v, p1v, pAb = train(trainMat, listClasses)
print("不当语句占比：", pAb)
print("每个单词在不当语句中出现的概率(对数形式)：", p1v)

不当语句占比： 0.5
每个单词在不当语句中出现的概率(对数形式)： [-3.23867845 -3.93182563 -3.23867845 -3.93182563 -3.93182563 -3.23867845
 -3.93182563 -3.93182563 -3.23867845 -3.93182563 -3.23867845 -3.23867845
 -3.23867845 -3.93182563 -3.93182563 -2.83321334 -3.93182563 -3.23867845
 -3.93182563 -3.93182563 -3.93182563 -3.23867845 -3.23867845 -2.54553127
 -3.93182563 -3.93182563 -3.93182563 -3.93182563 -2.83321334 -3.23867845
 -3.23867845 -3.93182563]


### **案例4：预测新语句所述的类别**

In [4]:
# 执行预测分类
def classify(vec2Classify, p0Vec, p1Vec, pAb):
    p1 = sum(vec2Classify * p1Vec) + np.log(pAb)
    p0 = sum(vec2Classify * p0Vec) + np.log(1 - pAb)
    if p1 > p0:                                               # 对比哪个类别可能性更高
        return 1
    else:
        return 0

testEntry1 = ['love', 'my', 'dalmation']                  # 待预测的语句，以单词数组形式存在
testEntry2 = ['stupid', 'garbage']
thisDoc = setOfWords2Vec(myVocabList, testEntry1)         # 按照单词表，构造本语句的单词向量形式
print(testEntry1, 'classified as:', classify(thisDoc, p0v, p1v, pAb))
thisDoc = setOfWords2Vec(myVocabList, testEntry2)
print(testEntry2, 'classified as:', classify(thisDoc, p0v, p1v, pAb))

['love', 'my', 'dalmation'] classified as: 0
['stupid', 'garbage'] classified as: 1


### **案例5：使用sklearn.naive_bayes.MultinomialNB进行文本感情色彩分析**

In [5]:
''' 使用MultinomialNB对文本进行分类 '''

import numpy as np
from sklearn.naive_bayes import MultinomialNB

def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec=[0, 1, 0, 1, 0, 1]
    return postingList,classVec

def createVocabList(dataSet):
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet | set(document)
    return list(vocabSet)

def setOfWords2Vec(vocabList,inputSet):
    returnVec = np.zeros(len(vocabList))          # 每个元素对应vocabList中的一个单词              
    for word in inputSet:
        if word in vocabList:
            # inputSet中的某个单词存在vocabList中，则returnVec中对应单词位置元素值设为1
            returnVec[vocabList.index(word)] = 1      
        else: print('单词【%s】在词汇表中暂不存在，忽略!'% word)
    return returnVec

listPosts, listClasses = loadDataSet()
myVocabList = createVocabList(listPosts)

trainMat=[]
for postinDoc in listPosts:
    trainMat.append(setOfWords2Vec(myVocabList, postinDoc))

model = MultinomialNB()
model.fit(trainMat, listClasses)
testEntry1 = ['love', 'my', 'dalmation']
testEntry2 = ['stupid', 'garbage']
testMat = []
testMat.append(setOfWords2Vec(myVocabList,testEntry1))
testMat.append(setOfWords2Vec(myVocabList,testEntry2))
print(model.predict(testMat))

[0 1]
